In [ ]:
!pip install tesseract-ocr
!pip install pdf2image
!pip install pytesseract 
!pip install poppler-utils

In [ ]:
!pip install numpy==1.19.3 --user

In [ ]:
!pip install pandas

In [ ]:
!pip install poppler

In [ ]:
from PIL import Image
import pytesseract
import os
import cv2
import numpy as np
from io import BytesIO
import pandas as pd
import glob
import csv
from pdf2image import convert_from_path

In [ ]:
img = convert_from_path('C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/data.pdf', 300)
#img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
for page in img:
  page.save('C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/data.jpg', 'JPEG')

In [ ]:
#====== PREPROCESSING AND TEXT SAVING
im = Image.open("C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/data.jpg")
img = cv2.imread("C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/data.jpg")
img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
# Convert to gray
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply dilation and erosion to remove some noise
kernel = np.ones((1, 1), np.uint8)
img = cv2.dilate(img, kernel, iterations=1)
img = cv2.erode(img, kernel, iterations=1)
# Apply blur to smooth out the edges
img = cv2.GaussianBlur(img, (5, 5), 0)
# Apply threshold to get image with only b&w (binarization)
img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
cv2.imwrite("C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/dataProcessed.jpg", img)

text = pytesseract.image_to_string(img, lang='eng')
text_file = open("C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/data.txt", "w")
n = text_file.write(text)
text_file.close()

In [ ]:
def pdfToImg(pdfPath):
    img = convert_from_path(pdfPath, 300)
    for page in img:
        page.save(pdfPath[:-4] + ".jpg", 'JPEG')
        print("Converted " + pdfPath + " to jpeg")

def preprocessImg(imgPath):
    img = cv2.imread(imgPath)
    # Resize to bigger format
    img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    # Apply blur to smooth out the edges
    img = cv2.GaussianBlur(img, (5, 5), 0)
    # Apply threshold to get image with only b&w (binarization)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cv2.imwrite(imgPath[:-4] + "Proc.jpg", img) 
    print("Preprocessed " + imgPath)
    return img

def ocrSaveText(img):   
    print("Converted " + img + " to text")
    text = pytesseract.image_to_string(img, lang='eng')
    text_file = open(img[:-4] + ".txt", "w")
    n = text_file.write(text)
    text_file.close()

#def saveText(text, textPath)
#    text_file = open(textPath[:-3] + ".txt", "w")
#    n = text_file.write(text)
#    text_file.close()

In [ ]:
path = "C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/pdf/*.*"
for file in glob.glob(path):
  #print(file)
  # convert pdf to img
  pdfToImg(file)
  preprocessImg(file[:-4] + ".jpg")
  ocrSaveText(file[:-4] + "Proc.jpg")
  #ocrSaveText(text, file[:-4])
  print("Conversion completed")

In [ ]:
#===== REGEX =====
# Info
lastNameRe = "[\n\r].*Last Name: \s*([^\n\r]\S*)"
firstNameRe = "[\n\r].*First Name: \s*([^\n\r]\S*)"
identificationRe = "[\n\r].*Identification: ([^\n\r]\d*\s\d*\s\d*)"
genderRe = "[\n\r].*Gender: \s*([^\n\r]\S*)"
dobRe = "[\n\r].*Date of Birth: \s*([^\n\r]\S*)"
ageRe = "[\n\r].*Age: \s*([^\n\r]\S*)"
heightRe = "[\n\r].*Height: \s*([^\n\r]\S*)"
weightRe = "[\n\r].*Weight: \s*([^\n\r]\S*)"
bmiRe = "[\n\r].*BMI: \s*([^\n\r]\S*)"
physicianRe = "[\n\r].*Physician: \s*([^\n\r].*)"
smokerRe = "[\n\r].*History: \s*([^\n\r]\S*)"
physiologistRe = "[\n\r].*Physiologist: \s*([^\n\r]\S*)"
clinicRe = "[\n\r].*Clinic: \s*([^\n\r]\S*)"

# Test 1
fevTest1Re = "FEV \S* \S* \S* \S* \S* ([^\n\r]\S*)"
fvcTest1Re = "FVC \S* \S* *\S* \S* ([^\n\r]\S*)"
vcMaxTest1Re = "VC MAX \S* \S* *\S* \S* ([^\n\r]\S*)"
fev1fvcTest1Re = "FEV 1% FVC % \S* \S* *\S* ([^\n\r]\S*)"
pefTest1Re = "PEF \S* \S* \S* \S* ([^\n\r]\S*)"
tlcTest1Re = "TLC \S* \S* \S* \S* ([^\n\r]\S*)"
vcTest1Re = "vc \S* \S* \S* \S* ([^\n\r]\S*)"
rvTest1Re = "RV \S* \S* \S* \S* ([^\n\r]\S*)"
rvTlcTest1Re = "RV % TLC \S* \S* \S* \S* ([^\n\r]\S*)"
dlcocTest1Re = "DLCOc \S* \S* \S* \S* ([^\n\r]\S*)"
vaTest1Re = "VA \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
kcocTest1Re = "KCOc \S* \S* \S* \S* ([^\n\r]\S*)"


# % Pred
fevPredRe = "FEV \w* \w* \S* \S* \S* \S* ([^\n\r]\S*)"
fvcPredRe = "FVC \S* \S* *\S* \S* \S* ([^\n\r]\S*)"
vcMaxPredRe = "VC MAX \S* \S* *\S* \S* \S* ([^\n\r]\S*)"
fev1fvcPredRe = "FEV 1% FVC % \S* \S* \S* \S* ([^\n\r]\S*)"
pefPredRe = "PEF \S* \S* \S* \S* \S* ([^\n\r]\S*)"
tlcPredRe = "TLC \S* \S* \S* \S* \S* ([^\n\r]\S*)"
vcPredRe = "vc \S* \S* \S* \S* \S* ([^\n\r]\S*)"
rvPredRe = "RV \S* \S* \S* \S* \S* ([^\n\r]\S*)"
rvTlcPredRe = "RV % TLC \S* \S* \S* \S* \S* ([^\n\r]\S*)"
dlcocPredRe = "DLCOc \S* \S* \S* \S* \S* ([^\n\r]\S*)"
vaPredRe = "VA \S* \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
kcocPredRe = "KCOc \S* \S* \S* \S* \S* ([^\n\r]\S*)"

# SR
fevSRRe = "FEV \w* \w* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
fvcSRRe = "FVC \S* \S* *\S* \S* \S* \S* ([^\n\r]\S*)"
vcMaxSRRe = "VC MAX \S* \S* *\S* \S* \S* \S* ([^\n\r]\S*)"
fev1fvcSRRe = "FEV 1% FVC % \S* \S* \S* \S* \S* ([^\n\r]\S*)"
pefSRRe = "PEF \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
tlcSRRe = "TLC \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
vcSRRe = "vc \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
rvSRRe = "RV \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
rvTlcSRRe = "RV % TLC \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
dlcocSRRe = "DLCOc \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"
kcocSRRe = "KCOc \S* \S* \S* \S* \S* \S* ([^\n\r]\S*)"


import re

# Info re.search
def regSearch(file):
    with open(file, 'r') as file:
        txt = file.read()
        try: 
            lastName = re.search(lastNameRe, txt)[1]
        except:
            lastName = "NA"

        try:
            firstName = re.search(firstNameRe, txt)[1]
        except: 
            firstName = "NA"
        
        try:
            identification = re.search(identificationRe, txt)[1]
        except:
            identification = "NA"
        
        try:
            gender = re.search(genderRe, txt)[1]     
        except:
            gender = "NA"

        try:
            dob = re.search(dobRe, txt)[1]
        except:
            dob = "NA"     

        try:
            age = re.search(ageRe, txt)[1]
        except:
            age = "NA"

        try:
            height = re.search(heightRe, txt)[1]
        except:
            height = "NA"

        try:
            weight = re.search(weightRe, txt)[1]
        except:
            weight = "NA"

        try:
            bmi = re.search(bmiRe, txt)[1]
        except:
            bmi = "NA"
        
        
        #physician      = re.search(physicianRe, txt)[1]
        #smoker         = re.search(smokerRe, txt)[1]
        #physiologist   = re.search(physiologistRe, txt)[1]
        #clinic         = re.search(clinicRe, txt)[1]

        # Test 1 re.search
        try:
            fevTest1     = re.search(fevTest1Re, txt)[1]
        except:
            fevTest1 = "NA"
        
        try:
            fvcTest1     = re.search(fvcTest1Re, txt)[1]
        except:
            fvcTest1 = "NA"
        
        try:
            vcMaxTest1   = re.search(vcMaxTest1Re, txt)[1]
        except:
            vcMaxTest1 = "NA"
        
        try:
            fev1fvcTest1 = re.search(fev1fvcTest1Re, txt)[1]
        except:
            fev1fvcTest1 = "NA"
        
        try:
            pefTest1     = re.search(pefTest1Re, txt)[1]
        except:
            pefTest1 = "NA"
        
        try:
            tlcTest1     = re.search(tlcTest1Re, txt)[1]
        except:
            tlcTest1 = "NA"
        
        try:
            vcTest1      = re.search(vcTest1Re, txt)[1]
        except:
            vcTest1 = "NA"
        
        try:
            rvTest1      = re.search(rvTest1Re, txt)[1]
        except:
            rvTest1 = "NA"
        
        try:
            rvTlcTest1   = re.search(rvTlcTest1Re, txt)[1]
        except:
            rvTlcTest1 = "NA"
        
        try:
            dlcocTest1   = re.search(dlcocTest1Re, txt)[1]
        except:
            dlcocTest1 = "NA"
        
        try:
            vaTest1      = re.search(vaTest1Re, txt)[1]
        except:
            vaTest1 = "NA"
        
        try:
            kcocTest1    = re.search(kcocTest1Re, txt)[1]
        except:
            kcocTest1 = "NA"
        
        # % Pred re.search
        try:
            fevPred     = re.search(fevPredRe, txt)[1]
        except:
            fevPred = "NA"
        
        try:
            fvcPred     = re.search(fvcPredRe, txt)[1]
        except:
            fvcPred = "NA"
        
        try:
            vcMaxPred   = re.search(vcMaxPredRe, txt)[1]
        except:
            vcMaxPred = "NA"
        
        try:
            fev1fvcPred = re.search(fev1fvcPredRe, txt)[1]
        except:
            fev1fvcPred = "NA"

        try:
            pefPred     = re.search(pefPredRe, txt)[1]
        except:
            pefPred = "NA"
        
        try:
            tlcPred     = re.search(tlcPredRe, txt)[1]
        except:
            tlcPred = "NA"
        
        try:
            vcPred      = re.search(vcPredRe, txt)[1] 
        except:
            vcPred = "NA"
        
        try:
            rvPred      = re.search(rvPredRe, txt)[1] 
        except:
            rvPred = "NA"
        
        try:
            rvTlcPred   = re.search(rvTlcPredRe, txt)[1]
        except:
            rvTlcPred = "NA"
        
        try:
            dlcocPred   = re.search(dlcocPredRe, txt)[1]
        except:
            dlcocPred = "NA"
        
        try:
            vaPred      = re.search(vaPredRe, txt)[1] 
        except:
            vaPred = "NA"
        
        try:
            kcocPred    = re.search(kcocPredRe, txt)[1]
        except:
            kcocPred = "NA"
        
        # SR re.search
        try:
            fevSR     = re.search(fevSRRe, txt)[1]
        except:
            fevSR = "NA"
        
        try:
            fvcSR     = re.search(fvcSRRe, txt)[1]
        except:
            fvcSR = "NA"
        
        try:
            vcMaxSR   = re.search(vcMaxSRRe, txt)[1]
        except:
            vcMaxSR = "NA"
        
        try:
            fev1fvcSR = re.search(fev1fvcSRRe, txt)[1]
        except:
            fev1fvcSR = "NA"
        
        try:
            pefSR     = re.search(pefSRRe, txt)[1]
        except:
            pefSR = "NA"
        
        try:
            tlcSR     = re.search(tlcSRRe, txt)[1]
        except:
            tlcSR = "NA"
        
        try:
            vcSR      = re.search(vcSRRe, txt)[1]
        except:
            vcSR = "NA"
        
        try:
            rvSR      = re.search(rvSRRe, txt)[1]
        except:
            rvSR = "NA"
        
        try:
            rvTlcSR   = re.search(rvTlcSRRe, txt)[1]   
        except:
            rvTlcSR = "NA"
        
        try:
            dlcocSR   = re.search(dlcocSRRe, txt)[1]
        except:
            dlcocSR = "NA"
        
        try:
            kcocSR    = re.search(kcocSRRe, txt)[1]
        except:
            kcocSR = "NA"
        
        writer.writerow([lastName, firstName, identification, gender, dob, age,
                    height, weight, bmi, fevTest1, fvcTest1, vcMaxTest1, fev1fvcTest1,
                    pefTest1, tlcTest1, vcTest1, rvTest1, rvTlcTest1,
                    dlcocTest1, vaTest1, kcocTest1, fevPred, fvcPred,
                    vcMaxPred, fev1fvcPred, pefPred, tlcPred, vcPred,
                    rvPred, rvTlcPred, dlcocPred, vaPred, kcocPred,
                    fevSR, fvcSR, vcMaxSR, fev1fvcSR, pefSR, tlcSR,
                    vcSR, rvSR, rvTlcSR, dlcocSR, kcocSR])

#INFO
#print("===INFO===")
#print(lastName)
#print(firstName)
#print(identification)
#print(gender)
#print(dob)
#print(age)
#print(height)
#print(weight)
#print(bmi)
#print(physician)
#print(smoker)
#print(physiologist)
#print(clinic)
##TEST
#print("===TEST1===")
#print(fevTest1)
#print(fvcTest1)
#print(vcMaxTest1)
#print(fev1fvcTest1)
#print(pefTest1)
#print(tlcTest1)
#print(vcTest1)
#print(rvTest1)
#print(rvTlcTest1)
#print(dlcocTest1)
#print(vaTest1)
#print(kcocTest1)
#
##PRED
#print("===PRED===")
#print(fevPred)
#print(fvcPred)
#print(vcMaxPred)
#print(fev1fvcPred)
#print(pefPred)
#print(tlcPred)
#print(vcPred)
#print(rvPred)
#print(rvTlcPred)
#print(dlcocPred)
#print(vaPred)
#print(kcocPred)
#
##SR
#print("===SR===")
#print(fevSR)
#print(fvcSR)
#print(vcMaxSR)
#print(fev1fvcSR)
#print(pefSR)
#print(tlcSR)
#print(vcSR)
#print(rvSR)
#print(rvTlcSR)
#print(dlcocSR)
#print(kcocSR)

In [ ]:
import csv
with open('C:/Users/Vin/Documents/Coding/patients-data-to-csv/data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["lastName", "firstName", "identification", "sex", "dob", "age",
                     "height", "weight", "bmi", "fevTest1", "fvcTest1", "vcMaxTest1", "fev1fvcTest1",
                     "pefTest1", "tlcTest1", "vcTest1", "rvTest1", "rvTlcTest1",
                     "dlcocTest1", "vaTest1", "kcocTest1", "fevPred", "fvcPred",
                     "vcMaxPred", "fev1fvcPred", "pefPred", "tlcPred", "vcPred",
                     "rvPred", "rvTlcPred", "dlcocPred", "vaPred", "kcocPred",
                     "fevSR", "fvcSR", "vcMaxSR", "fev1fvcSR", "pefSR", "tlcSR",
                     "vcSR", "rvSR", "rvTlcSR", "dlcocSR", "kcocSR"])
    path = "C:/Users/Vin/Documents/Coding/patients-data-to-csv/data/pdf/txt/*.*"
    for file in glob.glob(path):
        print(file)
        regSearch(file)